<a href="https://colab.research.google.com/github/Josepholaidepetro/tensorflow_job/blob/main/tfjob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf 
from sklearn.datasets import load_breast_cancer

In [2]:
# a quick neural network

In [3]:
import numpy as np
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.layers.core import Dropout
import logging
import sys
import argparse
np.random.seed(1671)

Note: The following parameters below are key in the training of the network

In [4]:
# network and training 
NB_EPOCH = 50
VERBOSE = 1
OPTIMIZER = Adam()
N_HIDDEN = 128

In [5]:
def input_fn():
  cancer = load_breast_cancer()
  X = cancer.data
  y = cancer.target
  y = np.expand_dims(y,axis=1)
  x = tf.cast(X, tf.float32)
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  dataset = dataset.repeat(100)
  dataset = dataset.batch(32)

  return dataset

In [6]:
def main(args):

  if len(args) < 2:
    print('You must specify model_dir for checkpoints such as'
          ' /tmp/tfkeras_example/.')
    return

  model_dir = args[1]
  print('Using %s to store checkpoints.' % model_dir)

  # Define a Keras Model.
  NB_CLASSES = 1 
  OPTIMIZER = Adam()
  N_HIDDEN = 128
  DROPOUT = 0.3

  model = Sequential()
  model.add(Dense(N_HIDDEN,input_shape=(30,)))
  model.add(Activation('relu'))
  model.add(Dropout(DROPOUT))
  model.add(Dense(N_HIDDEN))
  model.add(Activation('relu'))
  model.add(Dropout(DROPOUT))
  model.add(Dense(NB_CLASSES))
  model.add(Activation('sigmoid'))

  model.compile(loss='categorical_crossentropy',optimizer=OPTIMIZER,
              metrics=['accuracy'])
  
  model.summary()

  # Define DistributionStrategies and convert the Keras Model to an
  # Estimator that utilizes these DistributionStrateges.
  # Evaluator is a single worker, so using MirroredStrategy.
  

  config = tf.estimator.RunConfig(
          train_distribute=tf.distribute.MultiWorkerMirroredStrategy(),
          eval_distribute=tf.distribute.MirroredStrategy())
  keras_estimator = tf.keras.estimator.model_to_estimator(
      keras_model=model, config=config, model_dir=model_dir)

  # Train and evaluate the model. Evaluation will be skipped if there is not an
  # "evaluator" job in the cluster.

  tf.estimator.train_and_evaluate(
      keras_estimator,
      train_spec=tf.estimator.TrainSpec(input_fn=input_fn),
      eval_spec=tf.estimator.EvalSpec(input_fn=input_fn))


if __name__ == '__main__':
  logger = tf.get_logger()
  logger.setLevel(logging.INFO)
  tf.compat.v1.app.run(argv=sys.argv)

Using -f to store checkpoints.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               3968      
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)          

W0119 02:04:36.043182 139942307051392 collective_all_reduce_strategy.py:349] Collective ops is not configured at program startup. Some performance features may not be enabled.


INFO:tensorflow:Using MirroredStrategy with devices ('/device:CPU:0',)


I0119 02:04:36.050999 139942307051392 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/device:CPU:0',)


INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:CPU:0',), communication = CommunicationImplementation.AUTO


...

I0119 02:04:43.262053 139942307051392 estimator.py:2127] Saving 'checkpoint_path' summary for global step 1779: -f/model.ckpt-1779


INFO:tensorflow:Loss for final step: 0.0.


I0119 02:04:43.273416 139942307051392 estimator.py:350] Loss for final step: 0.0.


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
